In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType, DateType
import sys
import os
from delta import DeltaTable
from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
from delta.tables import *
import io
import json

In [0]:
def create_spark_session():
    return SparkSession \
        .builder \
        .appName("File Streaming Demo") \
        .master("local[3]") \
        .config("spark.databricks.delta.schema.autoMerge.enabled", "true")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .enableHiveSupport()\
        .getOrCreate()

In [0]:
def create_deltaTable_insert_update_rows(spark:SparkSession,columns:list, location:str,merge_condition:str,df:DataFrame):
    if (DeltaTable.isDeltaTable(spark, location)):
        print('tabela delta existente')
        deltaTable = DeltaTable.forPath(spark, location)
        deltaTable.alias('tgt') \
            .merge(
                df.alias('src'),
                merge_condition
            ) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
    else:
        print('tabela delta inexistente')    
        DeltaTable \
            .create(spark) \
            .addColumns(columns) \
            .location(location) \
            .execute()
        deltaTable = DeltaTable.forPath(spark, location)
        deltaTable.alias('tgt') \
            .merge(
                df.alias('src'),
                merge_condition
            ) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()


#### Caminhos para a camada silver

In [0]:
path_silver_dengue= '/FileStore/silver/dados_degue/casos_dengue'
path_silver_chuva= '/FileStore/silver/dados_degue/chuvas'

#### Leitura da dos dados em dataframes

In [0]:
df_dengue = spark.read.format('delta').load(path_silver_dengue)

In [0]:
df_chuva = spark.read.format('delta').load(path_silver_chuva)

#### Criação do banco de dados dengue_chuvas

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("Dengue_Chuvas").getOrCreate()

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS dengue_chuvas")

Out[9]: DataFrame[]

In [0]:
df_dengue.createOrReplaceTempView("tabela_dengue")
df_chuva.createOrReplaceTempView("tabela_chuva")

In [0]:
spark.sql("SHOW TABLES IN dengue_chuvas").display()

database,tableName,isTemporary
,tabela_chuva,true
,tabela_dengue,true


#### Analise soma casos dengue agrupado por 'ano','mes','estado','cidade'

In [0]:
df_dengue.groupBy('ano','mes','estado','cidade').agg(
    f.sum('quantidade_casos').alias('soma_casos_dengue')
    ).display()

ano,mes,estado,cidade,soma_casos_dengue
2015,12,ES,Cachoeiro De Itapemirim,1027.0
2019,7,ES,Cachoeiro De Itapemirim,1196.0
2016,4,ES,Conceição Da Barra,16.0
2015,5,ES,Conceição Do Castelo,2.0
2017,4,ES,Conceição Do Castelo,2.0
2015,5,ES,Divino De São Lourenço,0.0
2018,5,ES,Iúna,0.0
2015,4,ES,Jaguaré,18.0
2019,2,ES,Laranja Da Terra,27.0
2017,6,ES,Linhares,48.0


In [0]:
df_chuva.columns

Out[13]: ['data_medicao', 'mm', 'estado', 'dia', 'mes', 'ano']

In [0]:
df_dengue.columns

Out[14]: ['id',
 'data_medicao',
 'quantidade_casos',
 'codigo_ibge',
 'cidade',
 'estado',
 'cep',
 'latitude',
 'longitude',
 'ano',
 'mes',
 'dia']

#### Analise soma quantidade de chuva agrupado por 'ano','mes','estado'

In [0]:
df_media_chuva = df_chuva.groupBy('ano','mes','estado').agg(
    f.sum('mm').alias('soma_chuva')
)

In [0]:
df_media_chuva.display()

ano,mes,estado,soma_chuva
2018,6,GO,-2159784
2018,2,BA,-70199512
2018,3,SP,-20782533
2018,11,BA,-3875730
2018,11,RJ,-6854890
2018,6,AM,-74830411
2018,2,RN,-17377481
2018,4,PR,-21437379
2018,2,SP,-35052132
2018,9,TO,-33496286


#### Analise analise_chuvas_dengue 'ano','mes','estado'

In [0]:
df_media_dengue =  df_dengue.groupBy('ano','mes','estado').agg(
    f.sum('quantidade_casos').alias('soma_casos_degue')
)  #.filter((f.col('estado') == 'SP') & (f.col('ano') == '2015') & (f.col('mes') == '1'))
df_media_dengue.display()

ano,mes,estado,soma_casos_degue
2015,9,ES,3080.0
2017,2,PR,3456.0
2018,3,SP,481.0
2017,11,PR,2388.0
2018,11,RJ,507.0
2016,4,SP,6359.0
2017,7,ES,570.0
2018,2,SP,372.0
2018,4,PR,2446.0
2015,10,ES,3970.0


In [0]:
df_media_chuva = df_chuva.groupBy('ano','mes','estado').agg(
    f.abs(f.sum('mm')).alias('soma_chuva')
)#.filter((f.col('estado') == 'SP') & (f.col('ano') == '2015') & (f.col('mes') == '1'))
df_media_chuva.display()

ano,mes,estado,soma_chuva
2018,6,GO,2159784
2018,2,BA,70199512
2018,3,SP,20782533
2018,11,BA,3875730
2018,11,RJ,6854890
2018,6,AM,74830411
2018,2,RN,17377481
2018,4,PR,21437379
2018,2,SP,35052132
2018,9,TO,33496286


In [0]:
condicao = (df_media_dengue.ano == df_media_chuva.ano) & (df_media_dengue.mes == df_media_chuva.mes) & (df_media_dengue.estado == df_media_chuva.estado)
df_media_dengue.join(df_media_chuva,condicao,'inner').drop(df_media_dengue.ano, df_media_dengue.mes, df_media_dengue.estado).withColumn('casos_Dengue/ChuvaAcum',f.col('soma_casos_degue')/f.col('soma_chuva')).display()

soma_casos_degue,ano,mes,estado,soma_chuva,casos_Dengue/ChuvaAcum
481.0,2018,3,SP,20782533,2.314443576247419E-5
507.0,2018,11,RJ,6854890,7.39617995328882E-5
2446.0,2018,4,PR,21437379,1.1409976937945632E-4
372.0,2018,2,SP,35052132,1.0612763868400358E-5
1032.0,2018,12,SP,12814236,8.053542950200074E-5
0.0,2018,11,RS,17271851,0.0
1221.0,2018,8,ES,39267,0.031094812437924974
30.0,2018,7,MG,17298098,1.7342947184135505E-6
2084.0,2018,2,PR,36454131,5.716773223863161E-5
635.0,2018,9,CE,219965,0.0028868229036437616


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [0]:
resultado = df_media_dengue.join(df_media_chuva, condicao, 'inner')
resultado = resultado.withColumn("casos_Dengue/ChuvaAcum", F.col("soma_casos_degue") / F.col("soma_chuva"))

resultado = resultado.select(
    df_media_dengue['ano'],
    df_media_dengue['mes'],
    df_media_dengue['estado'],
    *[column for column in resultado.columns if column not in ["ano", "mes", "estado"]]
)

resultado.display()

ano,mes,estado,soma_casos_degue,soma_chuva,casos_Dengue/ChuvaAcum
2018,3,SP,481.0,20782533,2.314443576247419E-5
2018,11,RJ,507.0,6854890,7.39617995328882E-5
2018,4,PR,2446.0,21437379,1.1409976937945632E-4
2018,2,SP,372.0,35052132,1.0612763868400358E-5
2018,12,SP,1032.0,12814236,8.053542950200074E-5
2018,11,RS,0.0,17271851,0.0
2018,8,ES,1221.0,39267,0.031094812437924974
2018,7,MG,30.0,17298098,1.7342947184135505E-6
2018,2,PR,2084.0,36454131,5.716773223863161E-5
2018,9,CE,635.0,219965,0.0028868229036437616


In [0]:
# Salve o DataFrame resultante como uma tabela Delta gerenciada
resultado.write.format("delta").mode("overwrite").saveAsTable("analise_chuvas_dengue")

# Execute uma consulta SQL usando a tabela salva
spark.sql("SELECT * FROM analise_chuvas_dengue").display()

ano,mes,estado,soma_casos_degue,soma_chuva,casos_Dengue/ChuvaAcum
2018,3,SP,481.0,20782533,2.314443576247419E-5
2018,11,RJ,507.0,6854890,7.39617995328882E-5
2018,4,PR,2446.0,21437379,1.1409976937945632E-4
2018,2,SP,372.0,35052132,1.0612763868400358E-5
2018,12,SP,1032.0,12814236,8.053542950200074E-5
2018,11,RS,0.0,17271851,0.0
2018,8,ES,1221.0,39267,0.031094812437924974
2018,7,MG,30.0,17298098,1.7342947184135505E-6
2018,2,PR,2084.0,36454131,5.716773223863161E-5
2018,9,CE,635.0,219965,0.0028868229036437616
